# Create ArcGIS webmap on perceptual-model database

I referred to Jessica's instruction to create this notebook https://github.com/jlembury/GEOG594-Embury/blob/master/DEMO_ArcGIS_API_Python.ipynb

# Config

In [1]:
data_dir = f"..\data"

## Import modules

In [1]:
import os
import webbrowser
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import json
from arcgis.gis import GIS

## Connect with ArcGIS
Reference for authentication schemes: https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/

I ended up in using User autentification with OAuth 2.0 in Reference for authentication schemes: https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/

Note: log-in expires after an hour or so.
When commands return "Exception: A general error occurred: 'Response' object is not subscriptable", log-in again. 

In [2]:
with open('../auth.json', 'r') as infile:
    my_credentials = json.load(infile)

gis = GIS("https://sdsugeo.maps.arcgis.com/", client_id=my_credentials['client_id'])
print("Successfully logged in as: " + gis.properties.user.username)

# Clear the output for security 
from IPython.display import clear_output
clear_output(wait=False)

open

If you have ArcGIS pro, this is the most straightforward and easy connection

In [ ]:
# gis = GIS('Pro')
# print("Logged in as " + str(gis.properties.user.username))

## Newly create a webmap (skip the rest of the blocks if you are updating)

### Publish the CSV data to ArcGIS online

In [ ]:
# This section is to create a new webmap only
# Note: if this module says 'data already exist', skip to the section called 'Update to new data'
csv_path = os.path.join(data_dir, "giantTable.csv")
csv_properties={'title':'alltype_models_v1',
                'description':'Results of perceptual model analysis. Including Figure and Text models',
                'tags':'perceptual_model'} # Use different title for the new data 
csv_item = gis.content.add(item_properties=csv_properties, data=csv_path)

print(csv_item)

<Item title:"alltype_models_v1" type:CSV owner:raraki8159_SDSUGeo>


### Publish a feature layer from the CSV item

In [ ]:
# Note: this automatically converts latitude and longitude to point geometry
feature_layer_item = csv_item.publish()
print(feature_layer_item)
# Get feature layer id
feature_layer_id = feature_layer_item.id
print(feature_layer_id)
webbrowser.open('https://sdsugeo.maps.arcgis.com/apps/mapviewer/index.html?layers={}'.format(feature_layer_id), new=2)

In [ ]:
added_item = gis.content.get(feature_layer_id)

### Create a webmap from the feature layers 

In [ ]:
# Initialize map widget
webmap = gis.map()

# Set an map extent (global)
webmap.extent = {'xmin': -140,
                'ymin':-50,
                'xmax':180,
                'ymax':65,
                }

# Add the created layer
webmap.add_layer(added_item)

# Save the webmap
webmap_properties = {'title':'Perceptual Models Around the World v2.0',
                    'snippet': 'description',
                    'tags':'perceptual_model'}

webmap_item = webmap.save(webmap_properties)

# Display the webmap
webmap_item_id = webmap_item.id
print(webmap_item_id)
webbrowser.open('https://sdsugeo.maps.arcgis.com/apps/mapviewer/index.html?webmap={}'.format(webmap_item_id), new=2)
webmap_item

94bdbb06d8aa435598182f21c9ba067c


<Item title:"Perceptual Models Around the World v2.0" type:Web Map owner:raraki8159_SDSUGeo>

## Adjust display of webmap (need to run only after creating a new map)

This code is written for Updating webmap layer, due to weird behavior of ArcGIS. 

- Editing webmap vs. feature layer
  - Webmap can contain multiple feature layers, whereas feature layer is just a one gis layer.  
  - Webmap json will be pripritized over feature layer json, in terms of appearance setting (pop-up etc.) 

- What happens
  - When you first create webmap, you can only edit the appearance (pop-up etc.) of the feature layers. 
  - However, once you edit something and save on webmap, variable "['operationalLayers']" appears in the webmap json. After that, you'll only be able to edit the appearance of feature layer via "['operationalLayers']" in the webmap json file. 

In [ ]:
def get_map (map_id):
    '''
    GET MAP DATA FOR CHANGES
    '''
    
    m = gis.content.get(map_id)

    data = m.get_data()
    
    #Include the below line for prettified JSON
    #print(json.dumps(data, indent=4, sort_keys=True))

    print(m)
    
    return data
    
def update_map (map_id, data):
    '''
    UPDATE MAP TO SAVE CHANGES
    '''
    m = gis.content.get(map_id)
    
    # Set the item_properties to include the desired update
    properties = {"text": json.dumps(data)}

    # 'Commit' the updates to the Item
    update = m.update(item_properties=properties)
    
    return update

### Webmap update

In [ ]:
webmap_item_id = '94bdbb06d8aa435598182f21c9ba067c'
webmap_data = get_map(webmap_item_id)
print(json.dumps(webmap_data, indent=4, sort_keys=True))

<Item title:"Perceptual Models Around the World v2.0" type:Web Map owner:raraki8159_SDSUGeo>
{
    "authoringApp": "ArcGISPythonAPI",
    "authoringAppVersion": "2.0.1",
    "baseMap": {
        "baseMapLayers": [
            {
                "layerType": "ArcGISTiledMapServiceLayer",
                "resourceInfo": {
                    "capabilities": "Map,Tilemap,Query,Data",
                    "currentVersion": 10.3,
                    "exportTilesAllowed": false,
                    "fullExtent": {
                        "spatialReference": {
                            "latestWkid": 3857,
                            "wkid": 102100
                        },
                        "xmax": 20037507.067161843,
                        "xmin": -20037507.067161843,
                        "ymax": 19971868.88040863,
                        "ymin": -19971868.880408604
                    },
                    "initialExtent": {
                        "spatialReference": {
        

### Feature layer update

In [ ]:
feature_layer_id = '91f146af61004ce799233fc30f52a783'
feature_layer_data = get_map(feature_layer_id)
print(json.dumps(feature_layer_data, indent=4, sort_keys=True))

<Item title:"alltype_models_v1" type:Feature Layer Collection owner:raraki8159_SDSUGeo>
{
    "layers": [
        {
            "id": 0,
            "layerDefinition": {
                "defaultVisibility": true,
                "definitionExpression": null,
                "drawingInfo": {
                    "renderer": {
                        "field1": "model_type",
                        "type": "uniqueValue",
                        "uniqueValueGroups": [
                            {
                                "classes": [
                                    {
                                        "label": "Text model",
                                        "symbol": {
                                            "angle": 0,
                                            "contentType": "image/png",
                                            "height": 15,
                                            "imageData": "iVBORw0KGgoAAAANSUhEUgAAAHkAAAB5CAYAAAAd+o5JAAAAAXNSR0IArs4c

In [ ]:
# Update the basemaps

# US HUC8 layer 
HUC8_dict = {
                "id": "183343475b2-layer-50",
                "itemId": "988c850f4c944d53bd29bd934dd6ac22",
                "layerType": "ArcGISMapServiceLayer",
                "opacity": 0.25,
                "title": "Watershed Boundary Dataset (WBD) from NHDPlus Version 2.1 (Simplified/Nested), EPA OW",
                "url": "https://watersgeo.epa.gov/arcgis/rest/services/NHDPlus_NP21/WBD_NP21_Simplified/MapServer"
            }

# World waterbody maps
river_dict = {
                "id": "1830ae1b2b9-layer-53",
                "itemId": "9f86716d941c4410b0b406d911754b2c",
                "layerType": "ArcGISTiledMapServiceLayer",
                "title": "Esri Hydro Reference Overlay",
                "url": "https://tiles.arcgis.com/tiles/P3ePLMYs2RVChkJx/arcgis/rest/services/Esri_Hydro_Reference_Overlay/MapServer",
                "visibility": True
            }

webmap_data['baseMap']['baseMapLayers'].append(HUC8_dict)
webmap_data['baseMap']['baseMapLayers'].append(river_dict)

In [ ]:
# Make some changes

# Change the display names 
mylabel_dict = {0:{"fieldName":"id","label":"Model ID","visible":False},
                1:{"fieldName":"model_type","label":"Model type","visible":False},
                2:{"fieldName":"citation","label":"Citation","visible":False},
                3:{"fieldName":"url","label":"URL","visible":False},
                4:{"fieldName":"attribution","label":"Attribution","visible":False},
                5:{"fieldName":"attribution_url","label":"Attribution link","visible":False},
                6:{"fieldName":"figure_num","label":"Figure number","visible":False},
                7:{"fieldName":"figure_caption","label":"Figure caption","visible":False},
                8:{"fieldName":"figure_url","label":"Figure URL","visible":False},
                9:{"fieldName":"textmodel_snipped","label":"Text model snipped","visible":True},
                10:{"fieldName":"textmodel_section_number","label":"Text section number","visible":False},
                11:{"fieldName":"textmodel_section_name","label":"Text section name","visible":False},
                12:{"fieldName":"textmodel_page_number","label":"Text page number","visible":False},
                13:{"fieldName":"watershed_name","label":"Watershed name","visible":True},
                14:{"fieldName":"lat","label":"latitude","visible":False},
                15:{"fieldName":"lon","label":"longitude","visible":False},
                16:{"fieldName":"area_km2","label":"longitude","visible":False},
                17:{"fieldName":"num_spatial_zones","label":"Number of spatial zones","visible":True},
                18:{"fieldName":"spatial_property","label":"Spatial property","visible":True},
                19:{"fieldName":"num_temporal_zones","label":"Number of temporal zones","visible":True},
                20:{"fieldName":"temporal_property","label":"Temporal property","visible":True},
                21:{"fieldName":"vegetation_info","label":"Vegetation","visible":True},
                22:{"fieldName":"soil_info","label":"Soil","visible":True},
                23:{"fieldName":"geol_info","label":"Geology","visible":True},
                24:{"fieldName":"topo_info","label":"Topography","visible":True},
                25:{"fieldName":"three_d_info","label":"3-dimensional description","visible":True},
                26:{"fieldName":"uncertainty_info","label":"Uncertainty range","visible":True},
                27:{"fieldName":"other_info","label":"Information: other","visible":True},
                28:{"fieldName":"num_store","label":"Number of stores","visible":True},
                29:{"fieldName":"store_list","label":"List of stores","visible":True},
                30:{"fieldName":"store_id_list","label":"Hashtags of stores","visible":True},
                31:{"fieldName":"num_flux","label":"Number of fluxes","visible":True},
                32:{"fieldName":"flux_list","label":"List of fluxes","visible":True},
                33:{"fieldName":"flux_id_list","label":"Hashtags of fluxes","visible":True},
                34:{"fieldName":"dummy_column","label":"dummy_column","visible":False},
                35:{"fieldName":"huc_watershed_id","label":"US HUC8","visible":False},
                36:{"fieldName":"ObjectId","label":"ObjectID","visible":False}
                }

for i in range(len(mylabel_dict)): 
    if webmap_data['operationalLayers'][0]['popupInfo']['fieldInfos'][i]['fieldName'] == mylabel_dict[i]['fieldName']:
        print('The dictionary is in correct order')
        
        print(webmap_data['operationalLayers'][0]['popupInfo']['fieldInfos'][i]['label'])
        webmap_data['operationalLayers'][0]['popupInfo']['fieldInfos'][i]['label'] = mylabel_dict[i]['label']
        print(webmap_data['operationalLayers'][0]['popupInfo']['fieldInfos'][i]['label'])

        print(webmap_data['operationalLayers'][0]['popupInfo']['fieldInfos'][i]['visible'])
        webmap_data['operationalLayers'][0]['popupInfo']['fieldInfos'][i]['visible'] = mylabel_dict[i]['visible']
        print(webmap_data['operationalLayers'][0]['popupInfo']['fieldInfos'][i]['visible'])
    else:
        print('The dictionary is in wrong order')

The dictionary is in correct order
id
Model ID
True
False
The dictionary is in correct order
model_type
Model type
True
False
The dictionary is in correct order
citation
Citation
True
False
The dictionary is in correct order
url
URL
True
False
The dictionary is in correct order
attribution
Attribution
True
False
The dictionary is in correct order
attribution_url
Attribution link
True
False
The dictionary is in correct order
figure_num
Figure number
True
False
The dictionary is in correct order
figure_caption
Figure caption
True
False
The dictionary is in correct order
figure_url
Figure URL
True
False
The dictionary is in correct order
textmodel_snipped
Text model snipped
True
True
The dictionary is in correct order
textmodel_section_number
Text section number
True
False
The dictionary is in correct order
textmodel_section_name
Text section name
True
False
The dictionary is in correct order
textmodel_page_number
Text page number
True
False
The dictionary is in correct order
watershed_na

In [ ]:
# Turn off the layer name
webmap_data['operationalLayers'][0]['popupInfo']['title'] = ""

In [ ]:
# TODO: Add 'hand drawn figures' later 
render = {'type': 'uniqueValue',
      'visualVariables': [{'type': 'sizeInfo',
        'valueExpression': '$view.scale',
        'stops': [{'size': 46.875, 'value': 1155581.108577},
         {'size': 37.5, 'value': 9244648.868618},
         {'size': 18.75, 'value': 73957190.948944},
         {'size': 9.375, 'value': 591657527.591555}]}],
      'field1': 'model_type',
      'uniqueValueGroups': [{'classes': [{'label': 'Text model',
          'symbol': {'type': 'esriPMS',
           'angle': 0,
           'xoffset': 0,
           'yoffset': 0,
           'contentType': 'image/png',
           'imageData': 'iVBORw0KGgoAAAANSUhEUgAAAHkAAAB5CAYAAAAd+o5JAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAJcEhZcwAACxIAAAsSAdLdfvwAAAmoSURBVHhe7Z1Lbt82EMZzhBwhixyhB8g+geG8H02RtEGbNk3TBEX3WXSfI2TRA+QIXmTVduFFD+JFD/DvfAIV29I3FClxKIriB/xg2BalGQ4kDh+irjQ1bUL37j+6JtwQ3gsfhRPHIYK+zAcB58H5rrlLNOWWCwACgaCcCSxoqcD5cZ0u8M6EptSSyr0qPBc+3b338D/5yYKRBXf9TwLsuepMbJorV5GoUFrhhdAF3JncFCKpMLSvaBetH8Opgb2wu7XjmqRybty5++BPV2GzePT46eHrp88Oz7/9/vDdi5eHH16+Ovz06s0kL3983R2PciiP87Dzh+L8aO13r6Oj219JhSCxoRXmAwFBcBCoN7/+dnj77vdk4Hw4L86P67DrB3AC/5yr+5NUAB7L6O6wyqHgDsPdhspngbEG18X1Z9zp8HNfj3FxGN2RoDb3wcMnh2fPXxx+fv2WVvxawB7YBfuY3QT4+95VQb3Co0vaq38HzlMeP/mma1NTP4ZTA/tgJ+xlfgyB/9U+wm/fuf8Hc3oI2r+1HsdLgd2h7Tfqw1XN9nXz1tF1cep06OSQLQd3SESwT1E/rqq2KXHiWPC2vWjT8LhjlbV14FdAm436OXZVti2J4UiumFNfQKZaepu7FPgHP5n/A7aVlInB3q4RuiC1PJpDgb8BXa+PrgrLlRh5VbLHzwPDL4FuR+13rwb8hv+sXnpc/ZU58QHDBDXBqrntjSWgrUY9lhVoGOQMYwZ3DpU2mLE2qI/NBBqGOIOYod0gwV4fz1OgXiYGUcoItK8NbgGeZirQqF9X1etIjFCzaCQYzKkGZyIhWyfrlgur/eAW4HlMBDpvP1ouiJEsZkj36GEONMKYaKPzjIy5sWg6VNna4OVMtNFnWca65UI0k0Z3YEmASxCz66+//6F/twT16OlenbpQ2EibLkzRDy5Bml1rBNrXjzabpnRrsehFU4xklSCfXWsEGvXK6huYLDzAigZ2sVSZdAmasmuNQGsZN+LhQpNGclLaXcKsSqpEqwSF2JU70Khfz+xVmm6VnAirKmk2nXK6sASF2pU70KhnVv8C4rJ8FaichI5qYSKcGTSXEhRjV+5AexYeLBsN05Ktpd0lRgmKtStnoH3dqkVJmJyAvtlgMS9cgubYlTPQnmz7xIUsTlIQ7wCPTohViMyApZSguXblDLRnFWj8u1eSotOXz6zWZsWIlY9BU8yxQ+UKtJaEIV4udGGSQsioRyeyuotBjFj5GDTFHMuUK9Ceuzk805aD8Z7t6CRWdzGIESsfg6aYYzXlCLSnS/XBhXBacvCoX4wOObtgKmLEysegKeZYn3IEWhkgOXMh9EsOxBYOoxNYZNQXiRErH4OmmGOnZB1oT6Y9vbWFHDTaowP9M3ahlJSg1HZZBtrTb/7kQsklB2Dl5ahgjuU8JYjZVTKe5UL6Ck/5J31U51gzXYKYXSWDuLB4CfojW/45elRbJ1w9JYjZVTpKAqY/stlGaKknIjRKELOrdNjEBeLoQnpZ8k86jGnZN75ICWJ2lY6nzzwe5pQ/0oUB7MQW1Crma2pY3ITxggL542jGyXIYc0itYr6mRhnmHM9MyR9Ho1zYqIyd1IJaxXxNDeI0jJ1wefRL/kAnJHK1x6BWMV9T42mXzycs5BeadKVe/eGjVjFfU4M4sfgJ58mX/DJKunL1j3tqFfPVAqW/fJ58yS+jqcWcSReoVcxXC5Tk63zqUX4ZZda5BkF6ahXz1QJlNed5ho1fBv/MmlmDWsV8tUDJsC8FeXQANvpmJ7OiVjFfLdDml12IeZBzdp9ArWK+WqB1o1yItxdkVj4nMWLlLWhBTkyMWHkLWpATEyNW3oJZQf7lzTt6MitixMrnJEasvAXtTk5MjFh5C6oL8pbEfLWgBXlFMV8taEFeUcxXC2YF2fqNiSG1ivlqQciIVxu7NhLz1YKQses2C2Uk5qsFIbNQo81f2nxyGjFfLQiZT24rQ4zEfLUgZGVIW+NlJOZrakLXeLXVmkZivqZG6z4Jl7eXkD+0ddcGYr6mRsmsx7sOyB/bGxQGYr6mRkm66BsU7V0oAzFfU8PiJtB3oXb/VqOFmK8p8bTHfPO2vb+fbCHma0rYIIj6fjIkB+x6pwELMV9TovSP9Z0G5J+b2DOElc9JjFj5VMzdM2QTu/+w8jmJESufilm7/0ByAN3Hy3r0K0asfE5ixMqnAPGYtY8XJAdVvyPfUmLEyqdg0Y58kBw4Gv2y/jxfjFj5nMSIlU+BknCF7a0JycFF75K7JTFfl+LpG0ftklv0ftdbEvN1KcowJoj7skzJO9dvSczXJWh3cfTO9ZAU3N03KCzEfF2C5y6O/wYFJAVHM1PAItOuVczXuXgy6vGMU6j29l0oCzFf5+DpFy//OKecZDdfeLMQ83UOympMsOwLb5CcZDffarQQ8zUWT5cJcVn+rUZITrSLr65aiPkaA+pXGfgAab662gvf6yUXSTZ5UauYrzFokxCIhwtNOmlJGEiRbdcq5msonmza5kvoEL6lzy6IrC/HnPOeQH1q2TTi4EJiI7nI6fCiwKJbtVd83SXh1IXCTjdvHV2XC9FsGzNVLdDLQP2hHln9CmeofxcKW8nFjgcX/4L1lGTteAIMjl0I8kguSLtVIMdyoRrxLOcBabtLoZIL09Ew0AIdx0SAl49qLZH01z4TozpaGz3NRBuM/vBnV9XrSQzBCk+acYMWaJ2pAAuoV//Ky1yCIc4gZmjrRxN8/WBHOQHuBYOcYczgziGLeegtgnrYXIB7wTBfGw2QYOz18Q2/JxKsrg2Wn2UG+KLESDXrBphVsVorVirw1zOb1LNuFh0rMVjtR/dgIrz2uxr+eSb8L7JOP3ipxHCMjNEh0J6a2+qAthegfvKOZKWWG+tWE7IerEKs5REOPya6Rj2n2caic0ibphyy5WDDbs+S2UuYTxeuJUx0Y0UDc3oIkhQ87kpvs2Ef7AxIqjrgv9mEf0kSZ5GUedvqHrRp6HaUNpgCe2BXQJvbA3+3mVzNlTiMVaDertYQ3C3IVNd6nOO6uH7oXXsB+JlmVeUW5daO0Tc1pkD7h43KUPmpH+s4H86L84e2s4STXTyaQyUVckPaK/qSXSi4wxAQ3G0IDtpKBGoKHIfjUQ7lZ9ypl3B+zHs3aQ+SysFjHO9HB7XZBQF7Yfd+H8tzJBWGrS1Ge5gUBuwL28KhSZdUIma4uoCzDeVy4q7fBVYofyJhq5LKxbvT6IYhYbN+rOP8uA6u19rZtSSVj3a8DzzaRQQlNmPvy6B8F1Chta9NW9CVK/8Dprxw+rO3yFQAAAAASUVORK5CYII=',
           'url': 'https://static.arcgis.com/images/Symbols/Government/Warrant.png',
           'height': 15,
           'width': 15},
          'values': [['Text model']]},
         {'label': 'Figure model',
          'symbol': {'type': 'esriPMS',
           'angle': 0,
           'xoffset': 0,
           'yoffset': 0,
           'contentType': 'image/png',
           'imageData': 'iVBORw0KGgoAAAANSUhEUgAAAHkAAAB5CAYAAAAd+o5JAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAZdEVYdFNvZnR3YXJlAEFkb2JlIEltYWdlUmVhZHlxyWU8AAANaElEQVR4Xu2dz+ocSxXH5xHyCFncB3DhA+QRcsGFMYhXXIgIchERcZUr4kZuQlAQFMniigsRIiiCoAQMuhAuP0TuwoWKIOJCjKBwFy7G+aSmfulfzzmnqk6d6p6ZzBe+/Eimu7qqTp+/Vd29ueCC08D9x7d3vLPjgx2f7Phsz20D8zmPdqSdOzve3l/hgsXxSqAI5cWOktCiSPtcJwn+gkG4//jWjm/t+HTziUf/2f2VhLEM0/Wf7kh/bu17eIEbWbDSZB8Pk8AvaEDyr/jF0WY4mvSXfl/8uAr83b1339tPmI+f+s5285nvbzef++F28/kfbTdf+Ml288Wfl/n2T9PxnMf5tCO1X8s0jov/vsbHvvHR3YS0RsKJCAThIKivPNtuvvrrONIe7dI+15GuX+azl+N7bZHMMumONDky0TC0jcmXBDOaXJfrt2s643zNzHhKR+p87ie/vd189r3t5ku/kCd+LdIf+kX/pH4fkvE+2M/AGQPTde/hH2aDl/np7yafGm2Go0n/6GetdjP+szXhH3/36+Kg58T/rWWOe0m/a/0383E2ePOdN3aDujoY5JynLNw564V99XJ+Thr3H9/d0fa9+DTMnTRZp07GVfbZzM/d/YydGFJwJQ3qFYlUj93n9pLxMU5p/Dd5YkFZKTUiSDkX01xLxlsOzp7sZ/CIQcH+3sPns47fJGnHuWuvRsbN+KV5yUzzd6QLH2mlSA+wztn3trLsq5nHIxN0jYBXLGZ85Fvvb+987/cHvP3N34nHL0Lm42QEXRIwRY0FzPOtr/32peAe/Oqv26cf/HN79ff/blvw7M//fnke59OOdI1wMi/MjzRviUciaMsHDxYwGvroN39rFmgtEPzbP/vTWI0vCZr5XRVWFE2AIQ2qk0w42vaXf324F8Uy4EZC4FgMqV/dtAOylaJuKw8eIGBM6JP3/7Gf8nVBP4Zoty3ohfPoVMmSOpJMjzQAJxEuZvMYMUTYto9eqDKWatFyqTLQBzN5x6K5JeA+wsy47aNfLFPr1iJp0oHAIIuJ6wE+NEfL8O4PPnhpFTLf+vEfr3/DUvT6eM6nXWkszWQe9fTqai+JQdCWCwflwQiiFggVgfVMdE7DeiJ2zg3RaiuPHrZMmfZiyRcdWMmyBJ0j3lEpjjeSp1+kd1KbTWRepfmGQzYeaDs6BqVKU2J2p8BXh5nGSmLuWwLAFx/+L6aPWsSNPEKhpUusqgT6YY1oFGCSR2ltLRFciynHEkntVJP51VevgtKqtKtSjqYXXC5cW7hz4ibQ1hp0C5p5luY/ySVgF6hW1WIhXOpQIwlSMIUELNLvx0z6PnclGroFrW886KyGacFWR7o0Ferc7EnHnwLR6hp0CdpKq7qCMO3JhoZo2hLqHNL50cSvS//fS6LpGvPdFXXr0fazvcQakZ4BPmyQXYhSByasFeocUluRpF+Av9LvvUSApXSLG6ErvtB3gTqevdIePhOCLSLOXDnqwbzdaOb+4Uel3yOI5Srd3PwunVtFLQhDXk1IEfVhQ3stjhLqHAcDCiRaNsXIaB1Bl0x3V6Cpa3NDpJ2esz1sZHcX1dypXogDCuJ8sYObVDouijU+2l0s0VOqR3sJVkDKi0nIJxfiTozGtP1I5mLKFAhAOjaSCNEC/ls6r4pygeTFXoIFpFc4HDYgRNQEMDURZS3m7UcRrZXQnbtWULt2htui6JF2xastpHd0kJ9JF9oRLYky31L7vbT8Y5cmNdCKXegbfZTOM6nnzU/3klSQdl4enlixCBFhvqV2e4m2WnD7xQaiCJbFc2uzvl3I2OGpmerKteJe8y212ctS3joynZrSMttubUYukrxMky2Z6lnAVWKP+Zba62FJizNGplNTWjecW5vlAMww2dKL0BwLEVI0WwOprR7W5vGkV9L50bRuOrRZOqdIaeECOYrQyphChavEUkSpQWrLy1L6MofLXDpoabMr2tdzZqHMqW0MkBousOQHNUhtedlajXOby0Za2uyODyS5iRsKpBWnisWIOeflwxZI7XnocRfu4KeRVkoHXH2Qy5zCypRU5eJFZVKjBq1UijvVGqDUnofe/dqsC0vtRdPqn8tkI6e57A6qX9qChMMfW5E1dymarh0jtddKb9AHliqO5CVPCa4gUPfLkwULLehq3P2BEDXgI6fHSXfztC0vvVqc4dIkBzWL5rrRkJMkvxvBlxR0NebH0LpDJVPIhE4HO/+9lT1anLGUNlt7w1x5u5wvT4IvaWnREXRZqRNmWjqH/8/RuPR7C3u1OGMJbbb2hbl2rsjB12TpUYqsHUUQK22Rjs/EfPeWFyO0OGMJbbayEFc6J+/mnETYkpAdkXXWyDmm/ngUo7Q4Ywlt1uC64eUI+4aQDw/gRd9SYwY1uCLGBkZqccYS2qxlGS6l0NaXryH92Jg+WRM9upoUrcUZo/utuTeCUel4k1oadQ3px0YhW7XikUWGEVqcwWS7KlCVtApH0vEm1xbyyIX5UVqcMVKbrWxEOt7kuQoZLbPKpBEY6ZvXF/KXfyk3pnANIVuTFIlRkfZFkyuopWzRIAqWrt/Li5ALtEqoI6BV7Hp4VkIeEV2PDrjm6HqsRaE1Bul4k0sI2VqB4o6Vzunh6IBrjhEBmJYnu67lEnLDM8iZGqIrXlbddyRcq0MGtZhiVMXrpGrXRLtrwLU6ZFCDSykqaterrkK1cqnUaY5It2PFMK7rVKxCHb78ZaH1ZA/PQchrrCevsjPEy3MQsrUzxFUvr9gZcmf2Y2LjHi9rsSDSL1taMBKRlS8Nrsi6co/XYrs1pXNaafmzkYgq6lgWzxV0aenTweslFth3HaUJVk4+ElJfPLRMtcsfy5G18NYBKcJ2BF9WDhtZA7YsxghEuRvrBnVtFoBy0CU+QbHIs1BRJm9pvxwVOFpBo7toJMntRtCVoQVfDr9sDaR3V2bmyB0hc5R2iDCmmniDY6xyrEsBdH+svLxtoeeTo3Jmq8AfiVLqBLBepXFZN78rqoZSEUR9PhkEvGkg0wouovwbN9PohYqSFsMptOCypMXuoFTOj403DXS+M2TKUpoTVQce7Ztr+jkHFmZ+Y1hWp+ZGEul8Z4j77T8SrVo25sk1MIGjzHbtGrIEov9svks3vLuS5nr7D5BMtvP91qXBRS7ER6dULZGuBjQUS2NlG+6bHXm43uMFNJPtWF+GJS2LMttMVJRGt958PXCPv+uNfECqfjk/z8fkWwEHv0WuUKE51vUscJ5n0r3oCkDlgKvy3ZrAeEuueMECS8FRpH+GRN0tWo1w8YvePnjANemn1F6Rem7c9JZc833XHlopFcCnRgoa0h6pCQJHa7KG85d/Y5Yj3IUHXRU0uYwJG78s0/Dm+hoy4VYAAkYIegl6wFhdmqxpcfOb64FW5uzQZmvxIuMUBe2FKwbQtdjxDQogrUxBZ6QNMZ8loPGRwdho9qI6mtcjamHFqRYDvgsFa6pU3OVR68+jGYGi+dbz4oCPcw76wltt9Fu7wrMmo2Cab3k3Juz8whtIkfZh3gydQVhmraCPXaujcWC+9ZQJuQR8qxFoGwpIyDvMNqwx3RmkPVGbDiI5AtcBKPMrFz5g0FdXMwZ+PxktRVtrgbCPSbMjwdgozFzfzNoiRPj3k4EWhMGOaDuTaLqUR8/B8Zi2UZE4/rHGcvTgQKhT6tH0oC+hA76lL12QqM+x5jwn5qlUGdOQBY5gvOVCbhYsBLFCtiw1y4AtMIU6JfOpRdPIYSjuP746uCjsTKumRFAt5lsC5zOh3DRMqkaOgRo4RurjlBaqhTqllS4x/8Px5jtv7C4kR9usVAUJGq1GM9dGq5AJmLJFkY4tkvljHqX5Zd6Z/0Vw//Hd2cVf0bkkqbF1RSkaXFvq15RZqCH5vC5geHcvgYWgpVUwIOKeMwu714y3Al8v9WcI9e08MDhdqoVWDYMDBA3RFoIjzOJSWKSObgs4oKrVg3sPnwudSgz00RLRboopBFfRGo4GYzm4oULMsEbbB5MPP9/P9IpIOzzliBsOFvSUOQXCRxLV1mp7jrAJsnpSsGaWBJzmtbDzcimUBB2UR58VrTw48YgEnFEj6IDK2FmQeTg5AWfQMctHQwKMhcz30ZFx2wFW8sFHK+AprKgbsqrSuUx5cmS8+mpS5spRdCusPDqThfBz12rGpy/4T7lSHtyLVBmTS6CZ5+yry74XMj8LV7KikWrdekCWyS7EczHhjMNOjTKvlqtFLwFtmXLOUxY2/da3zN7k8OXCtcBCt7bDZE6CFMzdsfts+kc/y0FVIuMftuB/TEhBme2rM/FppB3HVkyhP/Sr7HMzGe+JBldepF2gdqo1J9pCpLqWOee6XL9Wa1+RcQbtqjxFpL1j8pMaJeL/eFEZkx9t1mmPdmm/1s8e8tnrYZprwbM82kN2tUTDEAjahnDwlQiqRI7jeM7j/HZNvck0DuezSa8Dkhnn+eg6n308pL/0+zU2yx6kV1scvsPkuEj/Kl/hcIGOtMKVBC69UG5JpusnwZ7EQsKpIj07TRpGwDbarNM+1+F6Fz+7GpIfv7MXBH4RobRG7Pkczk8CvfjXC04Dm83/AZ9peMb+hNWWAAAAAElFTkSuQmCC',
           'url': 'https://static.arcgis.com/images/Symbols/Government/Water-Teatment-Plant-Permit.png',
           'height': 18.75,
           'width': 18.75},
          'values': [['Figure model']]}]}],
      'uniqueValueInfos': [{'label': 'Text model',
        'symbol': {'type': 'esriPMS',
         'angle': 0,
         'xoffset': 0,
         'yoffset': 0,
         'contentType': 'image/png',
         'imageData': 'iVBORw0KGgoAAAANSUhEUgAAAHkAAAB5CAYAAAAd+o5JAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAJcEhZcwAACxIAAAsSAdLdfvwAAAmoSURBVHhe7Z1Lbt82EMZzhBwhixyhB8g+geG8H02RtEGbNk3TBEX3WXSfI2TRA+QIXmTVduFFD+JFD/DvfAIV29I3FClxKIriB/xg2BalGQ4kDh+irjQ1bUL37j+6JtwQ3gsfhRPHIYK+zAcB58H5rrlLNOWWCwACgaCcCSxoqcD5cZ0u8M6EptSSyr0qPBc+3b338D/5yYKRBXf9TwLsuepMbJorV5GoUFrhhdAF3JncFCKpMLSvaBetH8Opgb2wu7XjmqRybty5++BPV2GzePT46eHrp88Oz7/9/vDdi5eHH16+Ovz06s0kL3983R2PciiP87Dzh+L8aO13r6Oj219JhSCxoRXmAwFBcBCoN7/+dnj77vdk4Hw4L86P67DrB3AC/5yr+5NUAB7L6O6wyqHgDsPdhspngbEG18X1Z9zp8HNfj3FxGN2RoDb3wcMnh2fPXxx+fv2WVvxawB7YBfuY3QT4+95VQb3Co0vaq38HzlMeP/mma1NTP4ZTA/tgJ+xlfgyB/9U+wm/fuf8Hc3oI2r+1HsdLgd2h7Tfqw1XN9nXz1tF1cep06OSQLQd3SESwT1E/rqq2KXHiWPC2vWjT8LhjlbV14FdAm436OXZVti2J4UiumFNfQKZaepu7FPgHP5n/A7aVlInB3q4RuiC1PJpDgb8BXa+PrgrLlRh5VbLHzwPDL4FuR+13rwb8hv+sXnpc/ZU58QHDBDXBqrntjSWgrUY9lhVoGOQMYwZ3DpU2mLE2qI/NBBqGOIOYod0gwV4fz1OgXiYGUcoItK8NbgGeZirQqF9X1etIjFCzaCQYzKkGZyIhWyfrlgur/eAW4HlMBDpvP1ouiJEsZkj36GEONMKYaKPzjIy5sWg6VNna4OVMtNFnWca65UI0k0Z3YEmASxCz66+//6F/twT16OlenbpQ2EibLkzRDy5Bml1rBNrXjzabpnRrsehFU4xklSCfXWsEGvXK6huYLDzAigZ2sVSZdAmasmuNQGsZN+LhQpNGclLaXcKsSqpEqwSF2JU70Khfz+xVmm6VnAirKmk2nXK6sASF2pU70KhnVv8C4rJ8FaichI5qYSKcGTSXEhRjV+5AexYeLBsN05Ktpd0lRgmKtStnoH3dqkVJmJyAvtlgMS9cgubYlTPQnmz7xIUsTlIQ7wCPTohViMyApZSguXblDLRnFWj8u1eSotOXz6zWZsWIlY9BU8yxQ+UKtJaEIV4udGGSQsioRyeyuotBjFj5GDTFHMuUK9Ceuzk805aD8Z7t6CRWdzGIESsfg6aYYzXlCLSnS/XBhXBacvCoX4wOObtgKmLEysegKeZYn3IEWhkgOXMh9EsOxBYOoxNYZNQXiRErH4OmmGOnZB1oT6Y9vbWFHDTaowP9M3ahlJSg1HZZBtrTb/7kQsklB2Dl5ahgjuU8JYjZVTKe5UL6Ck/5J31U51gzXYKYXSWDuLB4CfojW/45elRbJ1w9JYjZVTpKAqY/stlGaKknIjRKELOrdNjEBeLoQnpZ8k86jGnZN75ICWJ2lY6nzzwe5pQ/0oUB7MQW1Crma2pY3ITxggL542jGyXIYc0itYr6mRhnmHM9MyR9Ho1zYqIyd1IJaxXxNDeI0jJ1wefRL/kAnJHK1x6BWMV9T42mXzycs5BeadKVe/eGjVjFfU4M4sfgJ58mX/DJKunL1j3tqFfPVAqW/fJ58yS+jqcWcSReoVcxXC5Tk63zqUX4ZZda5BkF6ahXz1QJlNed5ho1fBv/MmlmDWsV8tUDJsC8FeXQANvpmJ7OiVjFfLdDml12IeZBzdp9ArWK+WqB1o1yItxdkVj4nMWLlLWhBTkyMWHkLWpATEyNW3oJZQf7lzTt6MitixMrnJEasvAXtTk5MjFh5C6oL8pbEfLWgBXlFMV8taEFeUcxXC2YF2fqNiSG1ivlqQciIVxu7NhLz1YKQses2C2Uk5qsFIbNQo81f2nxyGjFfLQiZT24rQ4zEfLUgZGVIW+NlJOZrakLXeLXVmkZivqZG6z4Jl7eXkD+0ddcGYr6mRsmsx7sOyB/bGxQGYr6mRkm66BsU7V0oAzFfU8PiJtB3oXb/VqOFmK8p8bTHfPO2vb+fbCHma0rYIIj6fjIkB+x6pwELMV9TovSP9Z0G5J+b2DOElc9JjFj5VMzdM2QTu/+w8jmJESufilm7/0ByAN3Hy3r0K0asfE5ixMqnAPGYtY8XJAdVvyPfUmLEyqdg0Y58kBw4Gv2y/jxfjFj5nMSIlU+BknCF7a0JycFF75K7JTFfl+LpG0ftklv0ftdbEvN1KcowJoj7skzJO9dvSczXJWh3cfTO9ZAU3N03KCzEfF2C5y6O/wYFJAVHM1PAItOuVczXuXgy6vGMU6j29l0oCzFf5+DpFy//OKecZDdfeLMQ83UOympMsOwLb5CcZDffarQQ8zUWT5cJcVn+rUZITrSLr65aiPkaA+pXGfgAab662gvf6yUXSTZ5UauYrzFokxCIhwtNOmlJGEiRbdcq5msonmza5kvoEL6lzy6IrC/HnPOeQH1q2TTi4EJiI7nI6fCiwKJbtVd83SXh1IXCTjdvHV2XC9FsGzNVLdDLQP2hHln9CmeofxcKW8nFjgcX/4L1lGTteAIMjl0I8kguSLtVIMdyoRrxLOcBabtLoZIL09Ew0AIdx0SAl49qLZH01z4TozpaGz3NRBuM/vBnV9XrSQzBCk+acYMWaJ2pAAuoV//Ky1yCIc4gZmjrRxN8/WBHOQHuBYOcYczgziGLeegtgnrYXIB7wTBfGw2QYOz18Q2/JxKsrg2Wn2UG+KLESDXrBphVsVorVirw1zOb1LNuFh0rMVjtR/dgIrz2uxr+eSb8L7JOP3ipxHCMjNEh0J6a2+qAthegfvKOZKWWG+tWE7IerEKs5REOPya6Rj2n2caic0ibphyy5WDDbs+S2UuYTxeuJUx0Y0UDc3oIkhQ87kpvs2Ef7AxIqjrgv9mEf0kSZ5GUedvqHrRp6HaUNpgCe2BXQJvbA3+3mVzNlTiMVaDertYQ3C3IVNd6nOO6uH7oXXsB+JlmVeUW5daO0Tc1pkD7h43KUPmpH+s4H86L84e2s4STXTyaQyUVckPaK/qSXSi4wxAQ3G0IDtpKBGoKHIfjUQ7lZ9ypl3B+zHs3aQ+SysFjHO9HB7XZBQF7Yfd+H8tzJBWGrS1Ge5gUBuwL28KhSZdUIma4uoCzDeVy4q7fBVYofyJhq5LKxbvT6IYhYbN+rOP8uA6u19rZtSSVj3a8DzzaRQQlNmPvy6B8F1Chta9NW9CVK/8Dprxw+rO3yFQAAAAASUVORK5CYII=',
         'url': 'https://static.arcgis.com/images/Symbols/Government/Warrant.png',
         'height': 15,
         'width': 15},
        'value': 'Text model'},
       {'label': 'Figure model',
        'symbol': {'type': 'esriPMS',
         'angle': 0,
         'xoffset': 0,
         'yoffset': 0,
         'contentType': 'image/png',
         'imageData': 'iVBORw0KGgoAAAANSUhEUgAAAHkAAAB5CAYAAAAd+o5JAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAZdEVYdFNvZnR3YXJlAEFkb2JlIEltYWdlUmVhZHlxyWU8AAANaElEQVR4Xu2dz+ocSxXH5xHyCFncB3DhA+QRcsGFMYhXXIgIchERcZUr4kZuQlAQFMniigsRIiiCoAQMuhAuP0TuwoWKIOJCjKBwFy7G+aSmfulfzzmnqk6d6p6ZzBe+/Eimu7qqTp+/Vd29ueCC08D9x7d3vLPjgx2f7Phsz20D8zmPdqSdOzve3l/hgsXxSqAI5cWOktCiSPtcJwn+gkG4//jWjm/t+HTziUf/2f2VhLEM0/Wf7kh/bu17eIEbWbDSZB8Pk8AvaEDyr/jF0WY4mvSXfl/8uAr83b1339tPmI+f+s5285nvbzef++F28/kfbTdf+Ml288Wfl/n2T9PxnMf5tCO1X8s0jov/vsbHvvHR3YS0RsKJCAThIKivPNtuvvrrONIe7dI+15GuX+azl+N7bZHMMumONDky0TC0jcmXBDOaXJfrt2s643zNzHhKR+p87ie/vd189r3t5ku/kCd+LdIf+kX/pH4fkvE+2M/AGQPTde/hH2aDl/np7yafGm2Go0n/6GetdjP+szXhH3/36+Kg58T/rWWOe0m/a/0383E2ePOdN3aDujoY5JynLNw564V99XJ+Thr3H9/d0fa9+DTMnTRZp07GVfbZzM/d/YydGFJwJQ3qFYlUj93n9pLxMU5p/Dd5YkFZKTUiSDkX01xLxlsOzp7sZ/CIQcH+3sPns47fJGnHuWuvRsbN+KV5yUzzd6QLH2mlSA+wztn3trLsq5nHIxN0jYBXLGZ85Fvvb+987/cHvP3N34nHL0Lm42QEXRIwRY0FzPOtr/32peAe/Oqv26cf/HN79ff/blvw7M//fnke59OOdI1wMi/MjzRviUciaMsHDxYwGvroN39rFmgtEPzbP/vTWI0vCZr5XRVWFE2AIQ2qk0w42vaXf324F8Uy4EZC4FgMqV/dtAOylaJuKw8eIGBM6JP3/7Gf8nVBP4Zoty3ohfPoVMmSOpJMjzQAJxEuZvMYMUTYto9eqDKWatFyqTLQBzN5x6K5JeA+wsy47aNfLFPr1iJp0oHAIIuJ6wE+NEfL8O4PPnhpFTLf+vEfr3/DUvT6eM6nXWkszWQe9fTqai+JQdCWCwflwQiiFggVgfVMdE7DeiJ2zg3RaiuPHrZMmfZiyRcdWMmyBJ0j3lEpjjeSp1+kd1KbTWRepfmGQzYeaDs6BqVKU2J2p8BXh5nGSmLuWwLAFx/+L6aPWsSNPEKhpUusqgT6YY1oFGCSR2ltLRFciynHEkntVJP51VevgtKqtKtSjqYXXC5cW7hz4ibQ1hp0C5p5luY/ySVgF6hW1WIhXOpQIwlSMIUELNLvx0z6PnclGroFrW886KyGacFWR7o0Ferc7EnHnwLR6hp0CdpKq7qCMO3JhoZo2hLqHNL50cSvS//fS6LpGvPdFXXr0fazvcQakZ4BPmyQXYhSByasFeocUluRpF+Av9LvvUSApXSLG6ErvtB3gTqevdIePhOCLSLOXDnqwbzdaOb+4Uel3yOI5Srd3PwunVtFLQhDXk1IEfVhQ3stjhLqHAcDCiRaNsXIaB1Bl0x3V6Cpa3NDpJ2esz1sZHcX1dypXogDCuJ8sYObVDouijU+2l0s0VOqR3sJVkDKi0nIJxfiTozGtP1I5mLKFAhAOjaSCNEC/ls6r4pygeTFXoIFpFc4HDYgRNQEMDURZS3m7UcRrZXQnbtWULt2htui6JF2xastpHd0kJ9JF9oRLYky31L7vbT8Y5cmNdCKXegbfZTOM6nnzU/3klSQdl4enlixCBFhvqV2e4m2WnD7xQaiCJbFc2uzvl3I2OGpmerKteJe8y212ctS3joynZrSMttubUYukrxMky2Z6lnAVWKP+Zba62FJizNGplNTWjecW5vlAMww2dKL0BwLEVI0WwOprR7W5vGkV9L50bRuOrRZOqdIaeECOYrQyphChavEUkSpQWrLy1L6MofLXDpoabMr2tdzZqHMqW0MkBousOQHNUhtedlajXOby0Za2uyODyS5iRsKpBWnisWIOeflwxZI7XnocRfu4KeRVkoHXH2Qy5zCypRU5eJFZVKjBq1UijvVGqDUnofe/dqsC0vtRdPqn8tkI6e57A6qX9qChMMfW5E1dymarh0jtddKb9AHliqO5CVPCa4gUPfLkwULLehq3P2BEDXgI6fHSXfztC0vvVqc4dIkBzWL5rrRkJMkvxvBlxR0NebH0LpDJVPIhE4HO/+9lT1anLGUNlt7w1x5u5wvT4IvaWnREXRZqRNmWjqH/8/RuPR7C3u1OGMJbbb2hbl2rsjB12TpUYqsHUUQK22Rjs/EfPeWFyO0OGMJbbayEFc6J+/mnETYkpAdkXXWyDmm/ngUo7Q4Ywlt1uC64eUI+4aQDw/gRd9SYwY1uCLGBkZqccYS2qxlGS6l0NaXryH92Jg+WRM9upoUrcUZo/utuTeCUel4k1oadQ3px0YhW7XikUWGEVqcwWS7KlCVtApH0vEm1xbyyIX5UVqcMVKbrWxEOt7kuQoZLbPKpBEY6ZvXF/KXfyk3pnANIVuTFIlRkfZFkyuopWzRIAqWrt/Li5ALtEqoI6BV7Hp4VkIeEV2PDrjm6HqsRaE1Bul4k0sI2VqB4o6Vzunh6IBrjhEBmJYnu67lEnLDM8iZGqIrXlbddyRcq0MGtZhiVMXrpGrXRLtrwLU6ZFCDSykqaterrkK1cqnUaY5It2PFMK7rVKxCHb78ZaH1ZA/PQchrrCevsjPEy3MQsrUzxFUvr9gZcmf2Y2LjHi9rsSDSL1taMBKRlS8Nrsi6co/XYrs1pXNaafmzkYgq6lgWzxV0aenTweslFth3HaUJVk4+ElJfPLRMtcsfy5G18NYBKcJ2BF9WDhtZA7YsxghEuRvrBnVtFoBy0CU+QbHIs1BRJm9pvxwVOFpBo7toJMntRtCVoQVfDr9sDaR3V2bmyB0hc5R2iDCmmniDY6xyrEsBdH+svLxtoeeTo3Jmq8AfiVLqBLBepXFZN78rqoZSEUR9PhkEvGkg0wouovwbN9PohYqSFsMptOCypMXuoFTOj403DXS+M2TKUpoTVQce7Ztr+jkHFmZ+Y1hWp+ZGEul8Z4j77T8SrVo25sk1MIGjzHbtGrIEov9svks3vLuS5nr7D5BMtvP91qXBRS7ER6dULZGuBjQUS2NlG+6bHXm43uMFNJPtWF+GJS2LMttMVJRGt958PXCPv+uNfECqfjk/z8fkWwEHv0WuUKE51vUscJ5n0r3oCkDlgKvy3ZrAeEuueMECS8FRpH+GRN0tWo1w8YvePnjANemn1F6Rem7c9JZc833XHlopFcCnRgoa0h6pCQJHa7KG85d/Y5Yj3IUHXRU0uYwJG78s0/Dm+hoy4VYAAkYIegl6wFhdmqxpcfOb64FW5uzQZmvxIuMUBe2FKwbQtdjxDQogrUxBZ6QNMZ8loPGRwdho9qI6mtcjamHFqRYDvgsFa6pU3OVR68+jGYGi+dbz4oCPcw76wltt9Fu7wrMmo2Cab3k3Juz8whtIkfZh3gydQVhmraCPXaujcWC+9ZQJuQR8qxFoGwpIyDvMNqwx3RmkPVGbDiI5AtcBKPMrFz5g0FdXMwZ+PxktRVtrgbCPSbMjwdgozFzfzNoiRPj3k4EWhMGOaDuTaLqUR8/B8Zi2UZE4/rHGcvTgQKhT6tH0oC+hA76lL12QqM+x5jwn5qlUGdOQBY5gvOVCbhYsBLFCtiw1y4AtMIU6JfOpRdPIYSjuP746uCjsTKumRFAt5lsC5zOh3DRMqkaOgRo4RurjlBaqhTqllS4x/8Px5jtv7C4kR9usVAUJGq1GM9dGq5AJmLJFkY4tkvljHqX5Zd6Z/0Vw//Hd2cVf0bkkqbF1RSkaXFvq15RZqCH5vC5geHcvgYWgpVUwIOKeMwu714y3Al8v9WcI9e08MDhdqoVWDYMDBA3RFoIjzOJSWKSObgs4oKrVg3sPnwudSgz00RLRboopBFfRGo4GYzm4oULMsEbbB5MPP9/P9IpIOzzliBsOFvSUOQXCRxLV1mp7jrAJsnpSsGaWBJzmtbDzcimUBB2UR58VrTw48YgEnFEj6IDK2FmQeTg5AWfQMctHQwKMhcz30ZFx2wFW8sFHK+AprKgbsqrSuUx5cmS8+mpS5spRdCusPDqThfBz12rGpy/4T7lSHtyLVBmTS6CZ5+yry74XMj8LV7KikWrdekCWyS7EczHhjMNOjTKvlqtFLwFtmXLOUxY2/da3zN7k8OXCtcBCt7bDZE6CFMzdsfts+kc/y0FVIuMftuB/TEhBme2rM/FppB3HVkyhP/Sr7HMzGe+JBldepF2gdqo1J9pCpLqWOee6XL9Wa1+RcQbtqjxFpL1j8pMaJeL/eFEZkx9t1mmPdmm/1s8e8tnrYZprwbM82kN2tUTDEAjahnDwlQiqRI7jeM7j/HZNvck0DuezSa8Dkhnn+eg6n308pL/0+zU2yx6kV1scvsPkuEj/Kl/hcIGOtMKVBC69UG5JpusnwZ7EQsKpIj07TRpGwDbarNM+1+F6Fz+7GpIfv7MXBH4RobRG7Pkczk8CvfjXC04Dm83/AZ9peMb+hNWWAAAAAElFTkSuQmCC',
         'url': 'https://static.arcgis.com/images/Symbols/Government/Water-Teatment-Plant-Permit.png',
         'height': 18.75,
         'width': 18.75},
        'value': 'Figure model'}]}

webmap_data['operationalLayers'][0]['layerDefinition']['drawingInfo']['renderer'] = render
# feature_layer_data['layers'][0]['layerDefinition']['drawingInfo']['renderer']['uniqueValueInfos']
# feature_layer_data['layers'][0]['layerDefinition']['drawingInfo']['renderer']['field1'] == 'model_type' #['uniqueValueGroups'] #['uniqueValueGroups'] #['drawingInfo'] #['renderer']['symbol']['imageData']

# Update the icon to prettier one
# webmap_data['operationalLayers'][0]['layerDefinition']['drawingInfo']['renderer']['symbol']['url'] = "https://static.arcgis.com/images/Symbols/Government/Water-Teatment-Plant-Permit.png"
# webmap_data['operationalLayers'][0]['layerDefinition']['drawingInfo']['renderer']['symbol']['imageData'] = "iVBORw0KGgoAAAANSUhEUgAAAHkAAAB5CAYAAAAd+o5JAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAZdEVYdFNvZnR3YXJlAEFkb2JlIEltYWdlUmVhZHlxyWU8AAANaElEQVR4Xu2dz+ocSxXH5xHyCFncB3DhA+QRcsGFMYhXXIgIchERcZUr4kZuQlAQFMniigsRIiiCoAQMuhAuP0TuwoWKIOJCjKBwFy7G+aSmfulfzzmnqk6d6p6ZzBe+/Eimu7qqTp+/Vd29ueCC08D9x7d3vLPjgx2f7Phsz20D8zmPdqSdOzve3l/hgsXxSqAI5cWOktCiSPtcJwn+gkG4//jWjm/t+HTziUf/2f2VhLEM0/Wf7kh/bu17eIEbWbDSZB8Pk8AvaEDyr/jF0WY4mvSXfl/8uAr83b1339tPmI+f+s5285nvbzef++F28/kfbTdf+Ml288Wfl/n2T9PxnMf5tCO1X8s0jov/vsbHvvHR3YS0RsKJCAThIKivPNtuvvrrONIe7dI+15GuX+azl+N7bZHMMumONDky0TC0jcmXBDOaXJfrt2s643zNzHhKR+p87ie/vd189r3t5ku/kCd+LdIf+kX/pH4fkvE+2M/AGQPTde/hH2aDl/np7yafGm2Go0n/6GetdjP+szXhH3/36+Kg58T/rWWOe0m/a/0383E2ePOdN3aDujoY5JynLNw564V99XJ+Thr3H9/d0fa9+DTMnTRZp07GVfbZzM/d/YydGFJwJQ3qFYlUj93n9pLxMU5p/Dd5YkFZKTUiSDkX01xLxlsOzp7sZ/CIQcH+3sPns47fJGnHuWuvRsbN+KV5yUzzd6QLH2mlSA+wztn3trLsq5nHIxN0jYBXLGZ85Fvvb+987/cHvP3N34nHL0Lm42QEXRIwRY0FzPOtr/32peAe/Oqv26cf/HN79ff/blvw7M//fnke59OOdI1wMi/MjzRviUciaMsHDxYwGvroN39rFmgtEPzbP/vTWI0vCZr5XRVWFE2AIQ2qk0w42vaXf324F8Uy4EZC4FgMqV/dtAOylaJuKw8eIGBM6JP3/7Gf8nVBP4Zoty3ohfPoVMmSOpJMjzQAJxEuZvMYMUTYto9eqDKWatFyqTLQBzN5x6K5JeA+wsy47aNfLFPr1iJp0oHAIIuJ6wE+NEfL8O4PPnhpFTLf+vEfr3/DUvT6eM6nXWkszWQe9fTqai+JQdCWCwflwQiiFggVgfVMdE7DeiJ2zg3RaiuPHrZMmfZiyRcdWMmyBJ0j3lEpjjeSp1+kd1KbTWRepfmGQzYeaDs6BqVKU2J2p8BXh5nGSmLuWwLAFx/+L6aPWsSNPEKhpUusqgT6YY1oFGCSR2ltLRFciynHEkntVJP51VevgtKqtKtSjqYXXC5cW7hz4ibQ1hp0C5p5luY/ySVgF6hW1WIhXOpQIwlSMIUELNLvx0z6PnclGroFrW886KyGacFWR7o0Ferc7EnHnwLR6hp0CdpKq7qCMO3JhoZo2hLqHNL50cSvS//fS6LpGvPdFXXr0fazvcQakZ4BPmyQXYhSByasFeocUluRpF+Av9LvvUSApXSLG6ErvtB3gTqevdIePhOCLSLOXDnqwbzdaOb+4Uel3yOI5Srd3PwunVtFLQhDXk1IEfVhQ3stjhLqHAcDCiRaNsXIaB1Bl0x3V6Cpa3NDpJ2esz1sZHcX1dypXogDCuJ8sYObVDouijU+2l0s0VOqR3sJVkDKi0nIJxfiTozGtP1I5mLKFAhAOjaSCNEC/ls6r4pygeTFXoIFpFc4HDYgRNQEMDURZS3m7UcRrZXQnbtWULt2htui6JF2xastpHd0kJ9JF9oRLYky31L7vbT8Y5cmNdCKXegbfZTOM6nnzU/3klSQdl4enlixCBFhvqV2e4m2WnD7xQaiCJbFc2uzvl3I2OGpmerKteJe8y212ctS3joynZrSMttubUYukrxMky2Z6lnAVWKP+Zba62FJizNGplNTWjecW5vlAMww2dKL0BwLEVI0WwOprR7W5vGkV9L50bRuOrRZOqdIaeECOYrQyphChavEUkSpQWrLy1L6MofLXDpoabMr2tdzZqHMqW0MkBousOQHNUhtedlajXOby0Za2uyODyS5iRsKpBWnisWIOeflwxZI7XnocRfu4KeRVkoHXH2Qy5zCypRU5eJFZVKjBq1UijvVGqDUnofe/dqsC0vtRdPqn8tkI6e57A6qX9qChMMfW5E1dymarh0jtddKb9AHliqO5CVPCa4gUPfLkwULLehq3P2BEDXgI6fHSXfztC0vvVqc4dIkBzWL5rrRkJMkvxvBlxR0NebH0LpDJVPIhE4HO/+9lT1anLGUNlt7w1x5u5wvT4IvaWnREXRZqRNmWjqH/8/RuPR7C3u1OGMJbbb2hbl2rsjB12TpUYqsHUUQK22Rjs/EfPeWFyO0OGMJbbayEFc6J+/mnETYkpAdkXXWyDmm/ngUo7Q4Ywlt1uC64eUI+4aQDw/gRd9SYwY1uCLGBkZqccYS2qxlGS6l0NaXryH92Jg+WRM9upoUrcUZo/utuTeCUel4k1oadQ3px0YhW7XikUWGEVqcwWS7KlCVtApH0vEm1xbyyIX5UVqcMVKbrWxEOt7kuQoZLbPKpBEY6ZvXF/KXfyk3pnANIVuTFIlRkfZFkyuopWzRIAqWrt/Li5ALtEqoI6BV7Hp4VkIeEV2PDrjm6HqsRaE1Bul4k0sI2VqB4o6Vzunh6IBrjhEBmJYnu67lEnLDM8iZGqIrXlbddyRcq0MGtZhiVMXrpGrXRLtrwLU6ZFCDSykqaterrkK1cqnUaY5It2PFMK7rVKxCHb78ZaH1ZA/PQchrrCevsjPEy3MQsrUzxFUvr9gZcmf2Y2LjHi9rsSDSL1taMBKRlS8Nrsi6co/XYrs1pXNaafmzkYgq6lgWzxV0aenTweslFth3HaUJVk4+ElJfPLRMtcsfy5G18NYBKcJ2BF9WDhtZA7YsxghEuRvrBnVtFoBy0CU+QbHIs1BRJm9pvxwVOFpBo7toJMntRtCVoQVfDr9sDaR3V2bmyB0hc5R2iDCmmniDY6xyrEsBdH+svLxtoeeTo3Jmq8AfiVLqBLBepXFZN78rqoZSEUR9PhkEvGkg0wouovwbN9PohYqSFsMptOCypMXuoFTOj403DXS+M2TKUpoTVQce7Ztr+jkHFmZ+Y1hWp+ZGEul8Z4j77T8SrVo25sk1MIGjzHbtGrIEov9svks3vLuS5nr7D5BMtvP91qXBRS7ER6dULZGuBjQUS2NlG+6bHXm43uMFNJPtWF+GJS2LMttMVJRGt958PXCPv+uNfECqfjk/z8fkWwEHv0WuUKE51vUscJ5n0r3oCkDlgKvy3ZrAeEuueMECS8FRpH+GRN0tWo1w8YvePnjANemn1F6Rem7c9JZc833XHlopFcCnRgoa0h6pCQJHa7KG85d/Y5Yj3IUHXRU0uYwJG78s0/Dm+hoy4VYAAkYIegl6wFhdmqxpcfOb64FW5uzQZmvxIuMUBe2FKwbQtdjxDQogrUxBZ6QNMZ8loPGRwdho9qI6mtcjamHFqRYDvgsFa6pU3OVR68+jGYGi+dbz4oCPcw76wltt9Fu7wrMmo2Cab3k3Juz8whtIkfZh3gydQVhmraCPXaujcWC+9ZQJuQR8qxFoGwpIyDvMNqwx3RmkPVGbDiI5AtcBKPMrFz5g0FdXMwZ+PxktRVtrgbCPSbMjwdgozFzfzNoiRPj3k4EWhMGOaDuTaLqUR8/B8Zi2UZE4/rHGcvTgQKhT6tH0oC+hA76lL12QqM+x5jwn5qlUGdOQBY5gvOVCbhYsBLFCtiw1y4AtMIU6JfOpRdPIYSjuP746uCjsTKumRFAt5lsC5zOh3DRMqkaOgRo4RurjlBaqhTqllS4x/8Px5jtv7C4kR9usVAUJGq1GM9dGq5AJmLJFkY4tkvljHqX5Zd6Z/0Vw//Hd2cVf0bkkqbF1RSkaXFvq15RZqCH5vC5geHcvgYWgpVUwIOKeMwu714y3Al8v9WcI9e08MDhdqoVWDYMDBA3RFoIjzOJSWKSObgs4oKrVg3sPnwudSgz00RLRboopBFfRGo4GYzm4oULMsEbbB5MPP9/P9IpIOzzliBsOFvSUOQXCRxLV1mp7jrAJsnpSsGaWBJzmtbDzcimUBB2UR58VrTw48YgEnFEj6IDK2FmQeTg5AWfQMctHQwKMhcz30ZFx2wFW8sFHK+AprKgbsqrSuUx5cmS8+mpS5spRdCusPDqThfBz12rGpy/4T7lSHtyLVBmTS6CZ5+yry74XMj8LV7KikWrdekCWyS7EczHhjMNOjTKvlqtFLwFtmXLOUxY2/da3zN7k8OXCtcBCt7bDZE6CFMzdsfts+kc/y0FVIuMftuB/TEhBme2rM/FppB3HVkyhP/Sr7HMzGe+JBldepF2gdqo1J9pCpLqWOee6XL9Wa1+RcQbtqjxFpL1j8pMaJeL/eFEZkx9t1mmPdmm/1s8e8tnrYZprwbM82kN2tUTDEAjahnDwlQiqRI7jeM7j/HZNvck0DuezSa8Dkhnn+eg6n308pL/0+zU2yx6kV1scvsPkuEj/Kl/hcIGOtMKVBC69UG5JpusnwZ7EQsKpIj07TRpGwDbarNM+1+F6Fz+7GpIfv7MXBH4RobRG7Pkczk8CvfjXC04Dm83/AZ9peMb+hNWWAAAAAElFTkSuQmCC"

In [ ]:
# Change the pop-up
webmap_data['operationalLayers'][0]['popupInfo']['popupElements'] = [
                    {
                        "text": "<p><span style='font-size:18px;'>Perceptual model of <strong> {watershed_name}&nbsp;</strong></span></p>",
                        "type": "text"
                    },
                    {
                        "description": "",
                        "mediaInfos": [
                            {
                                "altText": "",
                                "caption": "<a href='{url}' rel='nofollow ugc' target='_blank'>{citation}</a>;&nbsp<a href='{attribution_url}' rel='nofollow ugc' target='_blank'>{attribution}</a>;&nbsp From Figure {figure_num};&nbsp Caption: {figure_caption}; Section {textmodel_section_number} &#8220{textmodel_section_name}&#8221 of pp.{textmodel_page_number}",
                                "refreshInterval": 0,
                                "title": "",
                                "type": "image",
                                "value": {
                                    "linkURL": "{url}",
                                    "sourceURL": "{figure_url}"
                                }
                            }
                        ],
                        "title": "",
                        "type": "media"
                    },
                    {
                        "text": "<span style='font-size:14px;'><strong>Text model</strong></span><p><span style='font-family:Times;font-size:10px;'>{textmodel_snipped}</span></p>",
                        "type": "text"
                    },
                    {
                        "text": "<p>\u25bc Process information in the model</p>",
                        "type": "text"
                    },
                    {
                        "description": "",
                        "fieldInfos": [
                            {
                                "fieldName": "num_store",
                                "isEditable": True,
                                "label": "Number of stores",
                                "visible": True
                            },
                            {
                                "fieldName": "store_list",
                                "isEditable": True,
                                "label": "List of stores",
                                "visible": True
                            },
                            {
                                "fieldName": "store_id_list",
                                "isEditable": True,
                                "label": "Hashtags of stores",
                                "visible": True
                            },
                            {
                                "fieldName": "num_flux",
                                "isEditable": True,
                                "label": "Number of fluxes",
                                "visible": True
                            },
                            {
                                "fieldName": "flux_list",
                                "isEditable": True,
                                "label": "List of fluxes",
                                "visible": True
                            },
                            {
                                "fieldName": "flux_id_list",
                                "isEditable": True,
                                "label": "Hashtags of fluxes",
                                "visible": True
                            }
                        ],
                        "title": "",
                        "type": "fields"
                    },
                    {
                        "text": "<p>\u25bc Spatio-temporal heterogeneity in the model</p>",
                        "type": "text"
                    },
                    {
                        "description": "",
                        "fieldInfos": [
                            {
                                "fieldName": "num_spatial_zones",
                                "isEditable": True,
                                "label": "Number of spatial zones",
                                "visible": True
                            },
                            {
                                "fieldName": "spatial_property",
                                "isEditable": True,
                                "label": "Spatial property",
                                "visible": True
                            },
                            {
                                "fieldName": "num_temporal_zones",
                                "isEditable": True,
                                "label": "Number of temporal zones",
                                "visible": True
                            },
                            {
                                "fieldName": "temporal_property",
                                "isEditable": True,
                                "label": "Temporal property",
                                "visible": True
                            }
                        ],
                        "title": "",
                        "type": "fields"
                    },
                    {
                        "text": "<p>\u25bc Ancillary information in the model</p>",
                        "type": "text"
                    },
                    {
                        "description": "",
                        "fieldInfos": [
                            {
                                "fieldName": "vegetation_info",
                                "isEditable": True,
                                "label": "Vegetation",
                                "visible": True
                            },
                            {
                                "fieldName": "soil_info",
                                "isEditable": True,
                                "label": "Soil",
                                "visible": True
                            },
                            {
                                "fieldName": "geol_info",
                                "isEditable": True,
                                "label": "Geology",
                                "visible": True
                            },
                            {
                                "fieldName": "topo_info",
                                "isEditable": True,
                                "label": "Topography",
                                "visible": True
                            },
                            {
                                "fieldName": "uncertainty_info",
                                "isEditable": True,
                                "label": "Uncertainty range",
                                "visible": True
                            },
                            {
                                "fieldName": "three_d_info",
                                "isEditable": True,
                                "label": "3-dimensional description",
                                "visible": True
                            }
                        ],
                        "title": "",
                        "type": "fields"
                    },
                    {
                        "text": "<p>\u25bc Watershed information</p>",
                        "type": "text"
                    },
                    {
                        "description": "",
                        "fieldInfos": [
                            {
                                "fieldName": "area_km2",
                                "isEditable": True,
                                "label": "Watershed area (km2)",
                                "visible": True
                            }
                        ],
                        "title": "",
                        "type": "fields"
                    }
                ]

In [ ]:
# # Update the map
# feature_layer_update = update_map(feature_layer_id, feature_layer_data)
# feature_layer_update

# Update the map
webmap_update = update_map(webmap_item_id, webmap_data)
webmap_update

True

In [ ]:
# Verify changes updated
webbrowser.open('https://sdsugeo.maps.arcgis.com/apps/mapviewer/index.html?webmap={}'.format(webmap_item_id), new=2)

# https://sdsugeo.maps.arcgis.com/apps/mapviewer/index.html?webmap=e535103892fd41cd915d60ca4d1d9d44

True